In [3]:
import numpy as np
import plotly.express as px
from pycaret.regression import *
from io import BytesIO
from azure.storage.blob import BlobServiceClient

In [4]:
# This is a temporaly line to run that adds the modules to the pythopath
sys.path.append('/home/azureuser/cloudfiles/code/Users/admin/Paychex_revenue_forecast')

NameError: name 'sys' is not defined

In [5]:
from src.paychex_ml.utils import load_credentials

ModuleNotFoundError: No module named 'src'

In [11]:
credentials = load_credentials("blob_storage",
                               file="/home/azureuser/cloudfiles/code/Users/admin/Paychex_revenue_forecast/credentials.yml")

In [6]:
# Global Variables
conn_string = credentials['conn_string']
account_key = credentials['account_key']
container = 'clean-data'
parquet_file = 'paychex_revenue.parquet'
#https://paychexdeveastus.blob.core.windows.net/clean-data/paychex_revenue.parquet

NameError: name 'credentials' is not defined

In [2]:
cols = ['Scenario'
       ,'Version'
       ,'Fiscal_Year'
       ,'11 Payroll Blended Products'
       ,'12 W2 Revenue'
       ,'13 Delivery Revenue'
       ,'14 ASO Allocation'
       ,'15 Other Processing Revenue'
       ,'16 SurePayroll'
       ,'17 Total International'
       ,'20 Total 401k'
       ,'31 HR Solutions (excl PEO)'
       ,'40 Total Online Services'
       ,'50 Other Managment Solutions'
       ,'60 Total PEO'
       ,'70 Total Insurance Services'
       ,'80 Interest on Funds Held for Clients'
       ,'Fiscal_Period'
       ,'Calendar_Date']

nums = ['11 Payroll Blended Products'
       ,'12 W2 Revenue'
       ,'13 Delivery Revenue'
       ,'14 ASO Allocation'
       ,'15 Other Processing Revenue'
       ,'16 SurePayroll'
       ,'17 Total International'
       ,'20 Total 401k'
       ,'31 HR Solutions (excl PEO)'
       ,'40 Total Online Services'
       ,'50 Other Managment Solutions'
       ,'60 Total PEO'
       ,'70 Total Insurance Services'
       ,'80 Interest on Funds Held for Clients']

def pad_months(mnth):
    mnth = str(mnth)
    if int(mnth) < 10:
        mnth = '0'+mnth
    return mnth 

def convert_months(mnth):
    #mnth = str(mnth)
    c_mnth = 'ERROR'
    if mnth == '01':
        c_mnth = '06'
    elif mnth == '02':
        c_mnth = '07'
    elif mnth == '03':
        c_mnth = '08'
    elif mnth == '04':
        c_mnth = '09'
    elif mnth == '05':
        c_mnth = '10'
    elif mnth == '06':
        c_mnth = '11'
    elif mnth == '07':
        c_mnth = '12'
    elif mnth == '08':
        c_mnth = '01'
    elif mnth == '09':
        c_mnth = '02'
    elif mnth == '10':
        c_mnth = '03'
    elif mnth == '11':
        c_mnth = '04'
    elif mnth == '12':
        c_mnth = '05'
    return c_mnth
        
# convert to calendar date
def convert_dates(df):
    df['Fiscal_Month'] = df.apply(lambda x: pad_months(x['Fiscal_Month']), axis=1)
    df['Calendar_Month'] = df.apply(lambda x: convert_months(x['Fiscal_Month']), axis=1) 
    df['f_yr'] = df['Fiscal_Year'].astype(str).str.replace('FY','')
    df['f_yr'] = df['f_yr'].astype(int)
    df['c_yr'] = np.where(df['Fiscal_Month'].astype(int) <= 7, df['f_yr'] - 1, df['f_yr'])
    df['Fiscal_Period'] = df['Fiscal_Year'] + df['Fiscal_Month']
    df['Calendar_Date'] = '20' + df['c_yr'].astype(str) + df['Calendar_Month'] + '01'
    df.drop(columns=['Fiscal_Month','Calendar_Month','f_yr','c_yr'], inplace=True)
    return df

blob_service_client = BlobServiceClient.from_connection_string(conn_string)
container_client = blob_service_client.get_container_client(container=container)

downloaded_blob = container_client.download_blob(parquet_file)
bytes_io = BytesIO(downloaded_blob.readall())
df = pd.read_parquet(bytes_io)
df = df.reset_index() # This unpacks the three level index that we currently have in the parquet file

# Actual DF
act_df = df.loc[df['Scenario'] == 'Actual']
act_df = act_df.iloc[:,4:]
act_df.columns = act_df.columns.str.replace(r'\d+', '')

# External data
conn_string = credentials['conn_string']
account_key = credentials['account_key']
container = 'external-data'
parquet_file = 'external_data.parquet'
#https://paychexdeveastus.blob.core.windows.net/clean-data/paychex_revenue.parquet

blob_service_client = BlobServiceClient.from_connection_string(conn_string)
container_client = blob_service_client.get_container_client(container=container)

downloaded_blob = container_client.download_blob(parquet_file)
bytes_io = BytesIO(downloaded_blob.readall())
edf = pd.read_parquet(bytes_io)
edf = edf.reset_index() # This unpacks the three level index that we currently have in the parquet file
edf=edf.sort_values(by=['date'])
edf=edf.rename(columns = {'date': 'Calendar Date'}, inplace = False)

#Join everything
full = pd.merge(act_df,edf,on='Calendar Date',how='left')

NameError: name 'BlobServiceClient' is not defined

In [1]:
act_df

NameError: name 'act_df' is not defined

In [15]:
#AUTO ARIMA

import pmdarima as pm
from pmdarima.arima import ARIMA
from pmdarima.arima.stationarity import ADFTest
from pmdarima.arima.utils import ndiffs


forecast_window=12

#Tests
for variable in act_df.columns: 
    adf_test = ADFTest(alpha=0.05)
    p_val, should_diff = adf_test.should_diff(act_df[variable]) 
    if p_val < 0.05:
        print('Time Series is stationary. p-value for ', variable, ' is ',  p_val)
    else:
        print('Time Series is not stationary. p-value for ' , variable,' is ',  p_val, '. Differencing is needed: ', should_diff)
    # Estimate the number of differences using an ADF test:
    n_adf = ndiffs(act_df[variable], test='adf') 
    print('n_adf:', n_adf)
    # Or a KPSS test (auto_arima default):
    n_kpss = ndiffs(act_df[variable], test='kpss') 
    print('n_kpss:', n_kpss)
    if n_adf == 1 & n_kpss == 1:
        print('Use differencing value of 1 when creating ARIMA model')    
    else:
        print('Differencing is not needed when creating ARIMA model')

                           
keys = []
tables = []

for variable in act_df.columns:
    stepwise_fit = pm.auto_arima(act_df[variable], start_p=1, start_q=1, max_p=5, max_q=5, m=12,
        start_P=0, seasonal=True, d=1, D=1, trace=True,
        error_action='ignore',  # don't want to know if an order does not work
        suppress_warnings=True,  # don't want convergence warnings
        stepwise=True)  # set to stepwise
    arima_table = pd.DataFrame(stepwise_fit.predict(n_periods=int(forecast_window)))
    keys.append(variable)
    tables.append(arima_table) 

df_arima = pd.concat(tables, keys=keys, axis=1)
df_arima.insert(loc = 0,
          column = 'Scenario',
          value = 'AutoArima')
df_arima.insert(loc = 1,
          column = 'Fiscal Period',
          value = pl_df.iloc[:, 1])
df_arima.insert(loc = 2,
          column = 'Calendar Date',
          value = pl_df.iloc[:, 2])

df_arima

Time Series is stationary. p-value for  Calendar Date  is  0.01
n_adf: 0
n_kpss: 1
Differencing is not needed when creating ARIMA model
Time Series is stationary. p-value for   Payroll Blended Products  is  0.01
n_adf: 0
n_kpss: 0
Differencing is not needed when creating ARIMA model
Time Series is stationary. p-value for   W Revenue  is  0.01
n_adf: 0
n_kpss: 0
Differencing is not needed when creating ARIMA model
Time Series is stationary. p-value for   Delivery Revenue  is  0.01
n_adf: 0
n_kpss: 0
Differencing is not needed when creating ARIMA model
Time Series is not stationary. p-value for   ASO Allocation  is  0.09393714497128472 . Differencing is needed:  True
n_adf: 1
n_kpss: 1
Use differencing value of 1 when creating ARIMA model
Time Series is stationary. p-value for   Other Processing Revenue  is  0.01
n_adf: 0
n_kpss: 0
Differencing is not needed when creating ARIMA model
Time Series is stationary. p-value for   SurePayroll  is  0.01
n_adf: 0
n_kpss: 1
Differencing is not nee

ValueError: Could not successfully fit a viable ARIMA model to input data.
See http://alkaline-ml.com/pmdarima/no-successful-model.html for more information on why this can happen.